<br>Let's talk about ***Decorators***.<br>

In [1]:
from functools import wraps, lru_cache

0x01 - the ```@wraps```

In [2]:
def eggs(func):
    
    @wraps(func)  # crucial line for deco (preserve the func's state)
    def _eggs(*args, **kwargs):
        ''' doc for deco '''
        print(f'{func.__name__!r} got args: {args!r} and ..')
        return func(*args, **kwargs)

    return _eggs

In [3]:
@eggs
def spam(a, b, c):
    ''' doc for spam '''
    return a * b * c

# its name, classname, ... are all preserved (by @wraps)
spam.__name__
help(spam)

'spam'

Help on function spam in module __main__:

spam(a, b, c)
    doc for spam



0x02 - *debug*

In [4]:
def debug(func):
    
    @wraps(func)
    def _debug(*args, **kwargs):
        ''' almost the same as the first example XD '''
        
        output = func(*args, **kwargs)
        print(f'{func.__name__!r} got args: {args!r}')
        
        return output
    return _debug 


@debug
def sayhi(times):
    return 'Hi ' * times

sayhi(5)

'sayhi' got args: (5,)


'Hi Hi Hi Hi Hi '

0x03 - *caching*

In [5]:
def memoize(func):
    ''' a similar func `lru_cache` introduce by Python in ver3.2 
    
        It (lru_cache) 
            contains a fixed cache size (128 by default) to save the memory
            and use some statistics to check whether the cache size should be increased
    '''
    
    func.cache = dict()  # access it by '__wrapped__.cache' in outer scope 
    
    @wraps(func)
    def _memoize(*args):
        
        if args not in func.cache:
            func.cache[args] = func(*args)
        
        return func.cache[args]
    return _memoize

In [6]:
# example: Fib

@memoize
def fib(n):
    if n < 2:
        return n
    else:
        return fib(n - 1) + fib(n - 2)
    
for i in range(1, 5):
    i, fib(i)
    
# check out the cache!
fib.__wrapped__.cache

(1, 1)

(2, 1)

(3, 2)

(4, 3)

{(1,): 1, (0,): 0, (2,): 1, (3,): 2, (4,): 3}

In [7]:
# example: how `lru_cache` works internally 

def counter(func):
    
    func.calls = 0
    
    @wraps(func)
    def _counter(*args, **kwargs):
        
        func.calls += 1
        
        return func(*args, **kwargs)
    return _counter


@lru_cache(maxsize=3)
@counter
def fib(n):
    if n < 2:
        return n 
    else:
        return fib(n - 1) + fib(n - 2)

In [8]:
fib(100)          # other num is fine as well :)
fib.cache_info()  # sort of statistics of caching 

fib.__wrapped__.__wrapped__.calls  # well, there's two decos ( normal for x.y.z )

354224848179261915075

CacheInfo(hits=98, misses=101, maxsize=3, currsize=3)

101

0x04 - deco with (optional) args

In [26]:
# well

def add(extra_n=1):
    ''' nesting! '''
    
    def _add(func):  
        ''' here's the real deco '''
        
        @wraps(func)
        def __add(n):
            return func(n + extra_n)  # e.g. sayhi(2+3)
        
        return __add
    return _add


# actual calls 
#   note: the result might be changed by the decorated version (sayhi)
add(extra_n=2)(sayhi)(2)
add(sayhi)(2)

'hi hi hi hi hi hi '

<function __main__.add.<locals>._add.<locals>.__add>

In [27]:
# example: call with decorators 

@add(extra_n=2)
def sayhi(n):
    return 'hi ' * n

sayhi(3)

'hi hi hi hi hi '

In [33]:
# example: 
#   check the deco was called with a function OR a regular param

def add(*args, **kwargs):
    ''' add n to the input of the decorated func '''
    
    default_kwargs = dict(n=1)
    
    def _add(func):
        
        @wraps(func)
        def __add(n):
            default_kwargs.update(kwargs)
            
            return func(n + default_kwargs['n'])
        return __add
    
    if len(args) == 1 and callable(args[0]) and not kwargs: 
        
        return _add(args[0])
    
    elif not args and kwargs:  
        
        default_kwargs.update(kwargs)
        return _add
    
    else:
        
        raise RuntimeError('This deco only supports kw args.')

In [38]:
@add
def hi(n):
    return 'Hi ' * n

hi(3)

@add(n=3)
def yo(n):
    return 'Yo ' * n

yo(3)

'Hi Hi Hi Hi '

'Yo Yo Yo Yo Yo Yo '